#Image Features and Homographies

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt

#Problem #1
Find the Hough transform (using the normal equation of a line) for the lines enclosing an object
with vertices A=(2,0), B=(2,2) and C=(0,2). Sketch the modified object enclosed by lines replacing
(rho,theta) of the given object lines with (rho*rho,theta + 90). Calculate the area of the modified
object.

In [ ]:
A = [2,0]
B = [0,2]
C = [0,2]

